In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import sys
sys.path.append(os.path.abspath('..'))
from utils.utils_vis import save_image

# Directory where the images are stored
save_dir = "/DataMount/NEPTUNE/Cropped_Vessels_Combined_Pred_Hyalinosis/"

# List all files in the directory
for img_name in os.listdir(save_dir):
    if not img_name.endswith("_pred_hya_20240706-234141_epoch_50.png"): continue
    # Construct the full path to the image
    img_path = os.path.join(save_dir, img_name)
    # Read the image as a grayscale image
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image is not completely black
    if np.any(img):
        # Find all contours in the original image
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Draw all contours
        # img_with_contours = cv2.drawContours(np.zeros_like(img), contours, -1, (255, 0, 0), 2)
        
        # Post-processing to remove small objects
        kernel = np.ones((3,3), np.uint8)
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=2)  # Remove small objects
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=2)  # Close small holes
        save_image(closing, img_path.replace(".png", "_processed.png"))
    else:
        shutil.copy(img_path, img_path.replace(".png", "_processed.png"))
        # # Find contours again after post-processing
        # processed_contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # img_with_processed_contours = cv2.drawContours(np.zeros_like(img), processed_contours, -1, (255, 0, 0), 2)
        
        # # Display original and processed images side by side
        # plt.figure(figsize=(12, 6))
        # plt.subplot(1, 2, 1)
        # plt.imshow(img_with_contours, cmap='gray')
        # plt.title('Original Contours')
        # plt.subplot(1, 2, 2)
        # plt.imshow(img_with_processed_contours, cmap='gray')
        # plt.title('Processed Contours')
        # plt.show()
